In [ ]:
import numpy as np
from PIL import Image
np.random.seed(0)  # 固定随机种子以确保结果可重复

m = 10  # 数据点的数量
n = 784  # 特征数量
lambda_ = 1e-5  # 正则化参数

true_x = np.array(Image.open('4.jpg'))
true_x = true_x.reshape(784)

x=true_x+np.random.randn(784)*(2.4e-3)

phi=np.random.randn(100,784)
O = np.random.randn(n, 100, 784)
A = [O[i] for i in range(m)]
y = [A[i] @ true_x for i in range(m)]

'''
Constant error setting:C=3 or 5 and Diminishing errors setting:C=20 or 30 and No error setting:C=0
Gaussian vector  e=C*r or C*r**t
'''
r=np.random.normal(loc=0, scale=1e-3, size=100)

print((np.linalg.norm((phi@(x-true_x)))**2)/len(y))  # start error


# PG Algorithms

In [2]:
#use by iterations
class pg_algorithms:

    def soft_thresholding(self, x, threshold):
        """Soft-thresholding operator for L1 norm."""
        return np.sign(x) * np.maximum(np.abs(x) - threshold, 0)

    def gradient_smooth_part(self, A, x, y):
        """Gradient of the smooth part of the objective function."""
        grad = np.zeros_like(x)
        n = len(y)
        for i in range(n):
            residual = A[i] @ x - y[i]
            grad += (A[i].T @ residual) / n
        return grad

    def proximal_gradient_descent(self, x, A, y, lambda_, true_x, r, phi_x, max_iter, p, alpha, C): 

        x0 = x.copy()
        y0 = y.copy()

        current_errs=[]
        current_errs.append((np.linalg.norm((phi_x@(x0-true_x)))**2)/len(y0))

       
        
        for iteration in range(max_iter):

            y0+=C*(r**(iteration+1))
            #y0+=C*r

            grad = self.gradient_smooth_part(A, x0, y0)
            x_new = self.soft_thresholding(x0 - alpha * grad, alpha * lambda_)
            x0 = x_new
            
            # 检查收敛性
            current_err = (np.linalg.norm((phi_x@(x0-true_x)))**2)/len(y0)
            if (iteration!=0) and (iteration % p == 0):
                current_errs.append(current_err)
                print(f"Iteration {iteration}, Objective Value: {current_err}")
        
            y0=y.copy()

        return x0,current_errs

In [3]:
import numpy as np
import time
#use by time
class pg_algorithms_time:

    def soft_thresholding(self, x, threshold):
        """Soft-thresholding operator for L1 norm."""
        return np.sign(x) * np.maximum(np.abs(x) - threshold, 0)

    def gradient_smooth_part(self, A, x, y):
        """Gradient of the smooth part of the objective function."""
        grad = np.zeros_like(x)
        n = len(y)
        for i in range(n):
            residual = A[i] @ x - y[i]
            grad += (A[i].T @ residual) / n
        return grad

    def proximal_gradient_descent(self, x, A, y, lambda_, true_x, r, phi_x, max_duration, p, alpha, C): 
        # 初始化变量
        start_time = time.time()  # 记录开始时间
        current_time = start_time
        next_print_time = start_time + p  # 下次打印的时间点
        print_interval = p  # 打印间隔
        iterations = 0  # 迭代计数器

        x0 = x.copy()
        y0 = y.copy()

        current_errs = []
        current_errs.append((np.linalg.norm((phi_x@(x0-true_x)))**2)/len(y0))

        while (current_time - start_time) < max_duration:  # 循环直到达到最大持续时间
            #y0+=C*(r**(iterations+1))
            y0 += C*r

            grad = self.gradient_smooth_part(A, x0, y0)
            x_new = self.soft_thresholding(x0 - alpha * grad, alpha * lambda_)
            x0 = x_new
            
            current_err = (np.linalg.norm((phi_x@(x0-true_x)))**2)/len(y0)
            iterations += 1

            # 检查是否达到下一次打印的时间点
            if current_time >= next_print_time:
                current_errs.append(current_err)
                print(f"Iteration {iterations}, Objective Value: {current_err}")
                next_print_time += print_interval  # 更新下次打印的时间点
            
            y0 = y.copy()
            current_time = time.time()  # 更新当前时间

        return x0, current_errs

# B-PG Algorithms

In [4]:
class bpg_algorithms:
#use by iterations
    def gradient_of_smooth_part(self, x, y, A):
        """计算平滑部分的梯度"""
        grad = np.zeros_like(x)
        for i in range(len(y)):
            residual = A[i] @ x - y[i]
            grad += 2 * A[i].T @ residual
        return grad / len(y)

    def soft_thresholding(self, x, lambda_):
        """软阈值操作 (用于L1正则化的近端步骤)"""
        return np.sign(x) * np.maximum(np.abs(x) - lambda_, 0)

    def block_proximal_gradient(self, x, A, y, lambda_, true_x, r, phi_x, max_iter, p, learning_rate, C): 
        """块近端梯度算法"""

        x0=x.copy()
        y0=y.copy()

        current_errs=[]
        current_errs.append((np.linalg.norm((phi_x@(x0-true_x)))**2)/len(y))
        for iteration in range(max_iter):

            y0+=C*(r**(iteration+1))
            #y0+=C*r

            # 计算平滑部分的梯度
            grad = self.gradient_of_smooth_part(x0, y0, A)
            
            # 近似更新（使用梯度下降）
            x_new = x0 - learning_rate * grad
            
            # 应用L1正则化的近端操作
            x_new = self.soft_thresholding(x_new, lambda_ * learning_rate)
            x0 = x_new
            
            # 检查收敛性
            current_err = (np.linalg.norm((phi_x@(x0-true_x)))**2)/len(y)
            if (iteration!=0) and (iteration % p == 0):
                current_errs.append(current_err)
                print(f"Iteration {iteration}, Objective Value: {current_err}")
            
            y0=y.copy()
            
        return x0,current_errs

In [5]:
import numpy as np
import time
#use by time
class bpg_algorithms_time:

    def gradient_of_smooth_part(self, x, y, A):
        """计算平滑部分的梯度"""
        grad = np.zeros_like(x)
        for i in range(len(y)):
            residual = A[i] @ x - y[i]
            grad += 2 * A[i].T @ residual
        return grad / len(y)

    def soft_thresholding(self, x, lambda_):
        """软阈值操作 (用于L1正则化的近端步骤)"""
        return np.sign(x) * np.maximum(np.abs(x) - lambda_, 0)

    def block_proximal_gradient(self, x, A, y, lambda_, true_x, r, phi_x, max_duration, p, learning_rate, C): 
        """块近端梯度算法"""

        start_time = time.time()  # 记录开始时间
        current_time = start_time
        next_print_time = start_time + p  # 下次打印的时间点
        print_interval = p  # 打印间隔
        iterations = 0  # 迭代计数器

        x0 = x.copy()
        y0 = y.copy()

        current_errs = []
        current_errs.append((np.linalg.norm((phi_x@(x0-true_x)))**2)/len(y))

        while (current_time - start_time) < max_duration:  # 循环直到达到最大持续时间
            y0+=C*(r**(iterations+1))
            #y0 += C*r

            # 计算平滑部分的梯度
            grad = self.gradient_of_smooth_part(x0, y0, A)
            
            # 近似更新（使用梯度下降）
            x_new = x0 - learning_rate * grad
            
            # 应用L1正则化的近端操作
            x_new = self.soft_thresholding(x_new, lambda_ * learning_rate)
            x0 = x_new
            
            # 检查收敛性
            current_err = (np.linalg.norm((phi_x@(x0-true_x)))**2)/len(y)
            

            # 检查是否达到下一次打印的时间点
            if current_time >= next_print_time:
                current_errs.append(current_err)
                print(f"Iteration {iterations}, Objective Value: {current_err}")
                next_print_time += print_interval  # 更新下次打印的时间点
            
            y0 = y.copy()
            iterations += 1  # 增加迭代计数器
            current_time = time.time()

        return x0, current_errs

# SPG Algorithms

In [6]:
class spg_algorithm:

    def prox_l1(self, v, lambda_):
        """Proximal operator for L1 regularization."""
        return np.sign(v) * np.maximum(np.abs(v) - lambda_, 0)

    def gradient_smooth_part(self, x, A, y, idx):
        """Compute the gradient of the smooth part with respect to a single sample or mini-batch."""
        return 2 *A[idx].T @ (A[idx] @ x - y[idx]) / len(y)

    def run_spg(self, x, A, y, lambda_, true_x, r, phi_x, num_iterations, p, step_size, C): 

        x0=x.copy()
        y0=y.copy()

        current_errs=[]
        current_errs.append((np.linalg.norm((phi_x@(x0-true_x)))**2)/len(y))
        for iteration in range(num_iterations):

            y0+=C*(r**(iteration+1))
            #y0+=C*r

            # Randomly select an index for stochastic gradient estimation
            idx = np.random.randint(len(y))
            
            # Compute the stochastic gradient of the smooth part
            grad_smooth = self.gradient_smooth_part(x0, A, y0, idx)
            
            # Perform a gradient descent step
            x_temp = x0 - step_size * grad_smooth
            
            # Apply the proximal operator corresponding to the non-smooth part
            x0 = self.prox_l1(x_temp, step_size * lambda_)
            
            # 检查收敛性
            current_err = (np.linalg.norm((phi_x@(x0-true_x)))**2)/len(y)
            if (iteration!=0) and (iteration % p == 0):
                current_errs.append(current_err)
                print(f"Iteration {iteration}, Objective Value: {current_err}")
        
            y0=y.copy()

        return x0,current_errs

In [7]:
import numpy as np
import time
 # by time
class spg_algorithm_time:

    def prox_l1(self, v, lambda_):
        """Proximal operator for L1 regularization."""
        return np.sign(v) * np.maximum(np.abs(v) - lambda_, 0)

    def gradient_smooth_part(self, x, A, y, idx):
        """Compute the gradient of the smooth part with respect to a single sample or mini-batch."""
        return 2 * A[idx].T @ (A[idx] @ x - y[idx]) / len(y)

    def run_spg(self, x, A, y, lambda_, true_x, r, phi_x, max_duration, p, step_size, C): 
        """Stochastic Proximal Gradient algorithm based on time iterations"""

        start_time = time.time()  # 记录开始时间
        current_time = start_time
        next_print_time = start_time + p  # 下次打印的时间点
        print_interval = p  # 打印间隔
        iterations = 0  # 迭代计数器

        x0 = x.copy()
        y0 = y.copy()

        current_errs = []
        current_errs.append((np.linalg.norm((phi_x@(x0-true_x)))**2)/len(y))

        while (current_time - start_time) < max_duration:  # 循环直到达到最大持续时间
            y0+=C*(r**(iterations+1))
            #y0 += C*r

            # Randomly select an index for stochastic gradient estimation
            idx = np.random.randint(len(y))
            
            # Compute the stochastic gradient of the smooth part
            grad_smooth = self.gradient_smooth_part(x0, A, y0, idx)
            
            # Perform a gradient descent step
            x_temp = x0 - step_size * grad_smooth
            
            # Apply the proximal operator corresponding to the non-smooth part
            x0 = self.prox_l1(x_temp, step_size * lambda_)
            
            # 检查收敛性
            current_err = (np.linalg.norm((phi_x@(x0-true_x)))**2)/len(y)
            

            # 检查是否达到下一次打印的时间点
            if current_time >= next_print_time:
                current_errs.append(current_err)
                print(f"Iteration {iterations}, Objective Value: {current_err}")
                next_print_time += print_interval  # 更新下次打印的时间点
            
            y0 = y.copy()
            iterations += 1  # 增加迭代计数器
            current_time = time.time()

        return x0, current_errs

# ADMN Algorithms

In [8]:
from scipy.sparse.linalg import cg
import math

class admm_algorithm:
    
    def soft_threshold(self, x, threshold):
        """Soft-thresholding operator."""
        return np.sign(x) * np.maximum(np.abs(x) - threshold, 0)

    def admm_solver(self, x, A, y, lambda_, true_x, r, phi_x, max_iter, p, rho, C):

        x0=x.copy()
        y0=y.copy()

        m, n = len(y0),len(x0)
        # 初始化变量
        z = x.copy()
        u = np.zeros(n)

        current_errs=[]
        current_errs.append((np.linalg.norm((phi_x@(x0-true_x)))**2)/len(y0)) 

        #y0+=C*r
        AtA = sum([A[i].T @ A[i] for i in range(m)]) / m + rho * np.eye(n)
        b1 = sum([A[i].T @ y0[i] for i in range(m)]) / m

        for iteration in range(max_iter):

            y0+=C*(r**(iteration+1))

            # x-update: 解决一个二次规划问题
            b = b1 + rho * (z - u)
            x0, _ = cg(AtA, b, x0, maxiter=10, tol=(1e-4/(iteration/2+1)/math.pow(0.999,(iteration+1)/3e2)*0.75 + 1e-9))
        
            # z-update: 使用软阈值操作
            z = self.soft_threshold(x0 + u, lambda_ / rho)

            # 对偶变量更新
            u = u + x0 - z

            current_err = (np.linalg.norm((phi_x@(x0-true_x)))**2)/len(y0)
            if (iteration!=0) and (iteration % p == 0):
                current_errs.append(current_err)
                print(f"Iteration {iteration}, Objective Value: {current_err}")

            y0=y.copy()

        return x0,current_errs

In [9]:
import numpy as np
import time
from scipy.sparse.linalg import cg
import math

class admm_algorithm_time:
    
    def soft_threshold(self, x, threshold):
        """Soft-thresholding operator."""
        return np.sign(x) * np.maximum(np.abs(x) - threshold, 0)

    def admm_solver(self, x, A, y, lambda_, true_x, r, phi_x, max_duration, p, rho, C):

        start_time = time.time()  # 记录开始时间
        current_time = start_time
        next_print_time = start_time + p  # 下次打印的时间点
        print_interval = p  # 打印间隔
        iterations = 0  # 迭代计数器

        x0 = x.copy()
        y0 = y.copy()

        m, n = len(y0), len(x0)
        # 初始化变量
        z = x.copy()
        u = np.zeros(n)

        current_errs = []
        current_errs.append((np.linalg.norm((phi_x@(x0-true_x)))**2)/len(y0)) 

        y0 += C*r
        AtA = sum([A[i].T @ A[i] for i in range(m)]) / m + rho * np.eye(n)
        b1 = sum([A[i].T @ y0[i] for i in range(m)]) / m

        while (current_time - start_time) < max_duration:  # 循环直到达到最大持续时间
            
            #y0+=C*(r**(iterations+1))

            # x-update: 解决一个二次规划问题
            b = b1 + rho * (z - u)
            x0, _ = cg(AtA, b, x0, maxiter=10, tol=(1e-4/(iterations/2+1)/math.pow(0.999,(iterations+1)/3e2)*0.75 + 1e-9))
        
            # z-update: 使用软阈值操作
            z = self.soft_threshold(x0 + u, lambda_ / rho)

            # 对偶变量更新
            u = u + x0 - z

            current_err = (np.linalg.norm((phi_x@(x0-true_x)))**2)/len(y0)
            

            # 检查是否达到下一次打印的时间点
            if current_time >= next_print_time:
                current_errs.append(current_err)
                print(f"Iteration {iterations}, Objective Value: {current_err}")
                next_print_time += print_interval  # 更新下次打印的时间点
            
            y0 = y.copy()
            iterations += 1  # 增加迭代计数器
            current_time = time.time()
            
        return x0, current_errs

# PGRR Algorithms

In [10]:
import numpy as np
from typing import List, Tuple

class pgrr_algorithm:
    def soft_thresholding(self, x: np.ndarray, threshold: float) -> np.ndarray:
        
        return np.sign(x) * np.maximum(np.abs(x) - threshold, 0)

    def compute_gradient(self, x: np.ndarray, A: List[np.ndarray], y: List[np.ndarray]) -> np.ndarray:
        
        n = len(y)
        gradient = np.zeros_like(x)
        for i in range(n):
            gradient += 2 * A[i].T @ (A[i] @ x - y[i])
        return gradient / n

    def PG_RR(self, initial_x, A, y, lambda_, true_x, r, phi_x, num_epochs, p, gamma, C):
       
        x = initial_x.copy()
        n = len(y)
        y0=y.copy()
        current_errs=[]
        current_errs.append((np.linalg.norm((phi_x@(x-true_x)))**2)/len(y))

        for iteration in range(num_epochs):

            y0+=C*(r**(iteration+1))
            #y0+=C*r

            for i in np.random.permutation(n):
                gradient = 2 * A[i].T @ (A[i] @ x - y0[i])
                x = self.soft_thresholding(x - gamma * gradient, gamma * lambda_)
            
            current_err = (np.linalg.norm((phi_x@(x-true_x)))**2)/len(y0)
            if (iteration!=0) and (iteration % p == 0):
                current_errs.append(current_err)
                print(f"Iteration {iteration}, Objective Value: {current_err}")
            
            y0=y.copy()
        
        return x,current_errs

In [11]:
import numpy as np
from typing import List, Tuple
import time

class pgrr_algorithm_time:
    def soft_thresholding(self, x: np.ndarray, threshold: float) -> np.ndarray:
        return np.sign(x) * np.maximum(np.abs(x) - threshold, 0)

    def compute_gradient(self, x: np.ndarray, A: List[np.ndarray], y: List[np.ndarray]) -> np.ndarray:
        n = len(y)
        gradient = np.zeros_like(x)
        for i in range(n):
            gradient += 2 * A[i].T @ (A[i] @ x - y[i])
        return gradient / n

    def PG_RR(self, initial_x, A, y, lambda_, true_x, r, phi_x, max_duration, p, gamma, C):
        start_time = time.time()  # 记录开始时间
        current_time = start_time
        next_print_time = start_time + p  # 下次打印的时间点
        print_interval = p  # 打印间隔
        iterations = 0  # 迭代计数器

        x = initial_x.copy()
        n = len(y)
        y0 = y.copy()
        current_errs = []
        current_errs.append((np.linalg.norm((phi_x@(x-true_x)))**2)/len(y))

        while (current_time - start_time) < max_duration:  # 循环直到达到最大持续时间
            
            #y0 += C*r
            y0+=C*(r**(iterations+1))
            # 对样本索引进行随机排列
            permuted_indices = np.random.permutation(n)
            
            for i in permuted_indices:
                gradient = 2 * A[i].T @ (A[i] @ x - y0[i])
                x = self.soft_thresholding(x - gamma * gradient, gamma * lambda_)
                
                # 检查是否达到下一次打印的时间点（在每次梯度更新后）
                
                if current_time >= next_print_time:
                    current_err = (np.linalg.norm((phi_x@(x-true_x)))**2)/len(y0)
                    current_errs.append(current_err)
                    print(f"Iteration {iterations}, Objective Value: {current_err}")
                    next_print_time += print_interval  # 更新下次打印的时间点
                
                iterations += 1  # 增加迭代计数器
                current_time = time.time()
                
            y0 = y.copy()

        return x, current_errs

# ours Algorithms

In [12]:
import numpy as np
import numpy as np

class ours_algorithms:

    def soft_thresholding(self ,x, threshold):
       
        return np.sign(x) * np.maximum(np.abs(x) - threshold, 0)
    
    def PG_RR(self, initial_x, A, y, lambda_, true_x, r, phi_x, num_epochs, p, gamma, C, momentum):

        x = initial_x.copy()
        n = len(y)
        velocity = np.zeros_like(x)
        y0=y.copy()

        current_errs=[]
        current_errs.append((np.linalg.norm((phi_x@(x-true_x)))**2)/len(y))
        
        
        for iteration in range(num_epochs):

            y0+=C*(r**(iteration+1))
            #y0+=C*r

            for i in np.random.permutation(n):
                gradient = 2 * A[i].T @ (A[i] @ x - y0[i])
                velocity = momentum * velocity + gamma * gradient
                x = self.soft_thresholding(x - velocity, gamma * lambda_)
        
            current_err = (np.linalg.norm((phi_x@(x-true_x)))**2)/len(y0)
            if (iteration!=0) and (iteration % p == 0):
                current_errs.append(current_err)
                print(f"Iteration {iteration}, Objective Value: {current_err}")
                
            y0=y.copy()

        return x,current_errs


In [13]:
import numpy as np
import time

class ours_algorithms_time:

    def soft_thresholding(self, x, threshold):
        return np.sign(x) * np.maximum(np.abs(x) - threshold, 0)

    def PG_RR(self, initial_x, A, y, lambda_, true_x, r, phi_x, max_duration, p, gamma, C, momentum):

        start_time = time.time()  # 记录开始时间
        current_time = start_time
        next_print_time = start_time + p  # 下次打印的时间点
        print_interval = p  # 打印间隔
        iterations = 0  # 迭代计数器

        x = initial_x.copy()
        n = len(y)
        velocity = np.zeros_like(x)
        y0 = y.copy()

        current_errs = []
        current_errs.append((np.linalg.norm((phi_x@(x-true_x)))**2)/len(y))

        while (current_time - start_time) < max_duration:  # 循环直到达到最大持续时间
            y0+=C*(r**(iterations+1))
            #y0 += C*r

            permuted_indices = np.random.permutation(n)
            
            for i in permuted_indices:
                gradient = 2 * A[i].T @ (A[i] @ x - y0[i])
                velocity = momentum * velocity + gamma * gradient
                x = self.soft_thresholding(x - velocity, gamma * lambda_)
                
                # 检查是否达到下一次打印的时间点（在每次梯度更新后）
                
                if current_time >= next_print_time:
                    current_err = (np.linalg.norm((phi_x@(x-true_x)))**2)/len(y0)
                    current_errs.append(current_err)
                    print(f"Iteration {iterations}, Objective Value: {current_err}")
                    next_print_time += print_interval  # 更新下次打印的时间点
                
                iterations += 1  # 增加迭代计数器
                current_time = time.time()
            y0 = y.copy()

        return x, current_errs

# plot

In [14]:
# PG algorithm
pg=pg_algorithms()

# B-PG algorithm
bpg = bpg_algorithms()

# SPG algorithm
spg = spg_algorithm()

# ADMM algorithm
admm = admm_algorithm()

# PGRR algorithm
pgrr=pgrr_algorithm()

# ours algorithm
ours = ours_algorithms()


# PG algorithm
pg_time=pg_algorithms_time()

# B-PG algorithm
bpg_time = bpg_algorithms_time()

# SPG algorithm
spg_time = spg_algorithm_time()

# ADMM algorithm
admm_time = admm_algorithm_time()

# PGRR algorithm
pgrr_time=pgrr_algorithm_time()

# ours algorithm
ours_time = ours_algorithms_time()

### No error

In [ ]:
C=0
p=5000
all_iteration=int(1.45e5+1)

pg_x,pg_errors= pg.proximal_gradient_descent(x, A, y, lambda_, true_x, r, phi, all_iteration, p, 2e-7, C)
bpg_x,bpg_errors = bpg.block_proximal_gradient(x, A, y, lambda_, true_x, r, phi, all_iteration, p, 1.1e-7, C)
spg_x,spg_errors= spg.run_spg(x, A, y, lambda_, true_x, r, phi, all_iteration, p, 1.4e-6, C)
admm_x,admm_errors = admm.admm_solver(x, A, y, lambda_, true_x, r, phi, all_iteration, p, 1e5, C)
pg_rr_x,pg_rr_errors= pgrr.PG_RR(x, A, y, lambda_, true_x, r, phi, all_iteration, p, 6.5e-8, C)
ours_x,ours_errors= ours.PG_RR(x, A, y, lambda_, true_x, r, phi, all_iteration, p, 6.5e-8, C, 0.9)

In [ ]:
max_duration = 55
p = 2
C = 0


pg_x,pg_errors= pg_time.proximal_gradient_descent(x, A, y, lambda_, true_x, r, phi, max_duration, p, 1.6e-7, C)#
bpg_x,bpg_errors = bpg_time.block_proximal_gradient(x, A, y, lambda_, true_x, r, phi, max_duration, p, 6e-8, C)#
spg_x,spg_errors= spg_time.run_spg(x, A, y, lambda_, true_x, r, phi, max_duration, p, 8.8e-8, C)#
admm_x,admm_errors = admm_time.admm_solver(x, A, y, lambda_, true_x, r, phi, max_duration, p, 7e5, C)
pg_rr_x,pg_rr_errors= pgrr_time.PG_RR(x, A, y, lambda_, true_x, r, phi, max_duration, p, 6.5e-8, C)#
ours_x,ours_errors= ours_time.PG_RR(x, A, y, lambda_, true_x, r, phi, max_duration, p, 6.5e-8, C, 0.9)#

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

#iterations = np.arange(0,all_iteration,p)
times = np.arange(0,max_duration,p)
fig, ax = plt.subplots(figsize=(6,5))

# 绘制数据
ax.plot(times, pg_errors, ls = '-.', label='PG')
ax.plot(times, bpg_errors, ls = '-', color = 'darkorange', marker = 's', label='B-PG')
ax.plot(times, spg_errors, ls = '-', color = 'forestgreen', marker = '^', label='SPG')
ax.plot(times, admm_errors, ls = '-', color = 'firebrick', marker = '*', label='ADMM')
ax.plot(times, pg_rr_errors, ls = '-', color = 'mediumpurple', marker = 'o', label='PG-RR')
ax.plot(times, ours_errors, ls = '-', color = 'dodgerblue', marker = 'h', label='Ours')

# 设置科学计数法
formatter = ticker.ScalarFormatter(useMathText=True)
formatter.set_scientific(True)
formatter.set_powerlimits((-1, 1))
ax.xaxis.set_major_formatter(formatter)

# 设置y轴从0开始
#ax.set_ylim(bottom=0,top=0.06)
ax.set_yscale('log')


# 移除上边框和右边框
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)


plt.xlabel("times")
plt.ylabel("Error")

# 添加图注
ax.legend(loc='best', fontsize=10)

# 调整布局以适应科学计数法的标签
plt.tight_layout()
plt.grid(ls = ':')
plt.show()

# 保存数组到.npy文件
# np.save('./data/name.npy', np.array([pg_errors,bpg_errors,spg_errors,admm_errors,pg_rr_errors,ours_errors]))

### Diminishing errors

In [ ]:
C=50
p=2000
all_iteration=int(5e4+1)

pg_x,pg_errors= pg.proximal_gradient_descent(x, A, y, lambda_, true_x, r, phi, all_iteration, p, 2e-7, C)
bpg_x,bpg_errors = bpg.block_proximal_gradient(x, A, y, lambda_, true_x, r, phi, all_iteration, p, 1.1e-7, C)
spg_x,spg_errors= spg.run_spg(x, A, y, lambda_, true_x, r, phi, all_iteration, p, 1.4e-6, C)
admm_x,admm_errors = admm.admm_solver(x, A, y, lambda_, true_x, r, phi, all_iteration, p, 1e5, C)
pg_rr_x,pg_rr_errors= pgrr.PG_RR(x, A, y, lambda_, true_x, r, phi, all_iteration, p, 6.5e-8, C)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

iterations = np.arange(0,all_iteration,p)

fig, ax = plt.subplots(figsize=(6,5))

# 绘制数据
ax.plot(iterations, pg_errors, ls = '-.', label='IPG')
ax.plot(iterations, bpg_errors, ls = '-', color = 'darkorange', marker = 's', label='IB-PG')
ax.plot(iterations, spg_errors, ls = '-', color = 'forestgreen', marker = '^', label='ISPG')
ax.plot(iterations, admm_errors, ls = '-', color = 'firebrick', marker = '*', label='IADMM')
ax.plot(iterations, pg_rr_errors, ls = '-', color = 'mediumpurple', marker = 'o', label='IPG-RR')

# 设置科学计数法
formatter = ticker.ScalarFormatter(useMathText=True)
formatter.set_scientific(True)
formatter.set_powerlimits((-1, 1))
ax.xaxis.set_major_formatter(formatter)

# 设置y轴从0开始
ax.set_ylim(bottom=0,top=0.06)


# 移除上边框和右边框
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)


plt.xlabel("Iterations")
plt.ylabel("Error")

# 添加图注
ax.legend(loc='best', fontsize=10)

# 调整布局以适应科学计数法的标签
plt.tight_layout()
plt.grid(ls = ':')
plt.show()

#np.save('./data/name.npy', np.array([pg_errors,bpg_errors,spg_errors,admm_errors]))

In [ ]:
max_duration = 55
p = 2
C = 50


pg_x,pg_errors= pg_time.proximal_gradient_descent(x, A, y, lambda_, true_x, r, phi, max_duration, p, 2.05e-7, C)#
bpg_x,bpg_errors = bpg_time.block_proximal_gradient(x, A, y, lambda_, true_x, r, phi, max_duration, p, 0.9e-7, C)#
spg_x,spg_errors= spg_time.run_spg(x, A, y, lambda_, true_x, r, phi, max_duration, p, 0.1e-6, C)#
admm_x,admm_errors = admm_time.admm_solver(x, A, y, lambda_, true_x, r, phi, max_duration, p, 1.1e6, C)
pg_rr_x,pg_rr_errors= pgrr_time.PG_RR(x, A, y, lambda_, true_x, r, phi, max_duration, p, 2.8e-8, C)#
ours_x,ours_errors= ours_time.PG_RR(x, A, y, lambda_, true_x, r, phi, max_duration, p, 6.5e-8, C, 0.9)#

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

#iterations = np.arange(0,all_iteration,p)
times = np.arange(0,max_duration,p)
fig, ax = plt.subplots(figsize=(6,5))

# 绘制数据
ax.plot(times, pg_errors, ls = '-.', label='PG')
ax.plot(times, bpg_errors, ls = '-', color = 'darkorange', marker = 's', label='B-PG')
ax.plot(times, spg_errors, ls = '-', color = 'forestgreen', marker = '^', label='SPG')
ax.plot(times, admm_errors, ls = '-', color = 'firebrick', marker = '*', label='ADMM')
ax.plot(times, pg_rr_errors, ls = '-', color = 'mediumpurple', marker = 'o', label='PG-RR')
ax.plot(times, ours_errors, ls = '-', color = 'dodgerblue', marker = 'h', label='Ours')


# 设置科学计数法
formatter = ticker.ScalarFormatter(useMathText=True)
formatter.set_scientific(True)
formatter.set_powerlimits((-1, 1))
ax.xaxis.set_major_formatter(formatter)

# 设置y轴从0开始

#ax.set_ylim(bottom=0,top=0.06)
ax.set_yscale('log')
#ax.yaxis.set_major_locator(ticker.LogLocator(base=10.0, numticks=15))  # 主刻度
ax.yaxis.set_minor_locator(ticker.LogLocator(base=10.0, subs='all'))


# 移除上边框和右边框
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.xlabel("times")
plt.ylabel("Error")

# 添加图注
ax.legend(loc='best', fontsize=10)

# 调整布局以适应科学计数法的标签
plt.tight_layout()
plt.grid(ls = ':')
plt.show()


# np.save('./data/name.npy', np.array([pg_errors,bpg_errors,spg_errors,admm_errors,pg_rr_errors,ours_errors]))

###  Constant error setting

In [ ]:
C=3  #5
p=5000
all_iteration=int(1.45e5+1)

pg_x,pg_errors= pg.proximal_gradient_descent(x, A, y, lambda_, true_x, r, phi, all_iteration, p, 2e-7, C)
bpg_x,bpg_errors = bpg.block_proximal_gradient(x, A, y, lambda_, true_x, r, phi, all_iteration, p, 1.1e-7, C)
spg_x,spg_errors= spg.run_spg(x, A, y, lambda_, true_x, r, phi, all_iteration, p, 1.4e-6, C)
admm_x,admm_errors = admm.admm_solver(x, A, y, lambda_, true_x, r, phi, all_iteration, p, 1e5, C)
pg_rr_x,pg_rr_errors= pgrr.PG_RR(x, A, y, lambda_, true_x, r, phi, all_iteration, p, 6.5e-8, C)
ours_x,ours_errors= ours.PG_RR(x, A, y, lambda_, true_x, r, phi, all_iteration, p, 6.5e-8, C, 0.9)

In [ ]:
max_duration = 55
p = 2
C = 3


pg_x,pg_errors= pg_time.proximal_gradient_descent(x, A, y, lambda_, true_x, r, phi, max_duration, p, 0.9e-7, C)#
bpg_x,bpg_errors = bpg_time.block_proximal_gradient(x, A, y, lambda_, true_x, r, phi, max_duration, p, 0.3e-7, C)#
spg_x,spg_errors= spg_time.run_spg(x, A, y, lambda_, true_x, r, phi, max_duration, p, 0.7e-7, C)#
admm_x,admm_errors = admm_time.admm_solver(x, A, y, lambda_, true_x, r, phi, max_duration, p, 1e7, C)
pg_rr_x,pg_rr_errors= pgrr_time.PG_RR(x, A, y, lambda_, true_x, r, phi, max_duration, p, 6e-9, C)#
ours_x,ours_errors= ours_time.PG_RR(x, A, y, lambda_, true_x, r, phi, max_duration, p, 6.5e-8, C, 0.9)#

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

#iterations = np.arange(0,all_iteration,p)
times = np.arange(0,max_duration,p)
fig, ax = plt.subplots(figsize=(6,5))

# 绘制数据
ax.plot(times, pg_errors, ls = '-.', label='PG')
ax.plot(times, bpg_errors, ls = '-', color = 'darkorange', marker = 's', label='B-PG')
ax.plot(times, spg_errors, ls = '-', color = 'forestgreen', marker = '^', label='SPG')
ax.plot(times, admm_errors, ls = '-', color = 'firebrick', marker = '*', label='ADMM')
ax.plot(times, pg_rr_errors, ls = '-', color = 'mediumpurple', marker = 'o', label='PG-RR')
ax.plot(times, ours_errors, ls = '-', color = 'dodgerblue', marker = 'h', label='Ours')

# 设置科学计数法
formatter = ticker.ScalarFormatter(useMathText=True)
formatter.set_scientific(True)
formatter.set_powerlimits((-1, 1))
ax.xaxis.set_major_formatter(formatter)

# 设置y轴从0开始
#ax.set_ylim(bottom=0,top=0.06)
ax.set_yscale('log')

# 移除上边框和右边框
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)


plt.xlabel("times")
plt.ylabel("Error")

# 添加图注
ax.legend(loc='best', fontsize=10)

# 调整布局以适应科学计数法的标签
plt.tight_layout()
plt.grid(ls = ':')
plt.show()

# np.save('./data/name.npy', np.array([pg_errors,bpg_errors,spg_errors,admm_errors,pg_rr_errors,ours_errors]))

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

#iterations = np.arange(0,all_iteration,p)
times = np.arange(0,max_duration,p)
fig, ax = plt.subplots(figsize=(6,5))

# 绘制数据
ax.plot(times, pg_errors, ls = '-.', label='PG')
ax.plot(times, bpg_errors, ls = '-', color = 'darkorange', marker = 's', label='B-PG')
ax.plot(times, spg_errors, ls = '-', color = 'forestgreen', marker = '^', label='SPG')
ax.plot(times, admm_errors, ls = '-', color = 'firebrick', marker = '*', label='ADMM')
ax.plot(times, pg_rr_errors, ls = '-', color = 'mediumpurple', marker = 'o', label='PG-RR')
ax.plot(times, ours_errors, ls = '-', color = 'dodgerblue', marker = 'h', label='Ours')

# 设置科学计数法
formatter = ticker.ScalarFormatter(useMathText=True)
formatter.set_scientific(True)
formatter.set_powerlimits((-1, 1))
ax.xaxis.set_major_formatter(formatter)

# 设置y轴从0开始
#ax.set_ylim(bottom=0,top=0.06)
ax.set_yscale('log')

# 移除上边框和右边框
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)


plt.xlabel("times")
plt.ylabel("Error")

# 添加图注
ax.legend(loc='best', fontsize=10)

# 调整布局以适应科学计数法的标签
plt.tight_layout()
plt.grid(ls = ':')
plt.show()

# np.save('./data/name.npy', np.array([pg_errors,bpg_errors,spg_errors,admm_errors,pg_rr_errors,ours_errors]))